In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dbdmobile/myanimelist-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.80G/1.80G [00:21<00:00, 91.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dbdmobile/myanimelist-dataset/versions/5


In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
df=pd.read_csv('/root/.cache/kagglehub/datasets/dbdmobile/myanimelist-dataset/versions/5/anime-filtered.csv')
df.head()

,anime_id,Name,Score,Genres,English name,Japanese name,sypnopsis,Type,Episodes,Aired,...,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,"In the year 2071, humanity has colonized sever...",TV,26,"Apr 3, 1998 to Apr 24, 1999",...,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,"other day, another bounty—such is the life of ...",Movie,1,"Sep 1, 2001",...,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,"Vash the Stampede is the man with a $$60,000,0...",TV,26,"Apr 1, 1998 to Sep 30, 1998",...,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),ches are individuals with special powers like ...,TV,26,"Jul 2, 2002 to Dec 24, 2002",...,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,It is the dark century and the people are suff...,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108


In [4]:
df.shape

(14952, 25)

In [5]:
# I'll select some of the relevant features for the recommendation

selected_features = ['Genres','sypnopsis','Type','Aired']
print(selected_features)

['Genres', 'sypnopsis', 'Type', 'Aired']


In [6]:
# replacing the null valuess with null string
for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [7]:
# combining all the 4 selected features
combined_features = df['Genres']+' '+df['sypnopsis']+' '+df['Type']+' '+df['Aired']
print(combined_features)


0        Action, Adventure, Comedy, Drama, Sci-Fi, Spac...
1        Action, Drama, Mystery, Sci-Fi, Space other da...
2        Action, Sci-Fi, Adventure, Comedy, Drama, Shou...
3        Action, Mystery, Police, Supernatural, Drama, ...
4        Adventure, Fantasy, Shounen, Supernatural It i...
                               ...                        
14947    Adventure, Mystery, Supernatural No synopsis i...
14948    Comedy, Horror, Supernatural ko is a typical h...
14949    Mystery, Dementia, Horror, Psychological, Supe...
14950    Adventure, Slice of Life, Comedy New Yama no S...
14951    Action, Fantasy Solar calendar year 2020: grot...
Length: 14952, dtype: object


In [8]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 796996 stored elements and shape (14952, 42905)>
  Coords	Values
  (0, 991)	0.05712891518018408
  (0, 1147)	0.0322777760291825
  (0, 7064)	0.04815058966893768
  (0, 10182)	0.03414676701498587
  (0, 32859)	0.03404243508132166
  (0, 12380)	0.03403748922607379
  (0, 35271)	0.09810755841338956
  (0, 17389)	0.08272548379092265
  (0, 37798)	0.20272616898763382
  (0, 41955)	0.043925299841272476
  (0, 360)	0.11182196101859564
  (0, 16753)	0.05978093763873593
  (0, 15509)	0.030521202010623416
  (0, 7005)	0.09502032550558621
  (0, 33495)	0.06319884642474102
  (0, 26588)	0.06700951224327671
  (0, 28635)	0.07713260692821101
  (0, 2044)	0.14673416811691498
  (0, 24586)	0.10503951945357264
  (0, 35010)	0.16497507904768668
  (0, 36805)	0.13468563403469716
  (0, 21459)	0.06805829268293671
  (0, 26266)	0.04284560740089697
  (0, 39683)	0.09850697989914324
  (0, 36527)	0.07506805207832812
  :	:
  (14951, 10120)	0.09257006041816418
  (14951, 15

In [9]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

print(similarity)

[[1.         0.27364235 0.11914773 ... 0.00920449 0.02822247 0.08639165]
 [0.27364235 1.         0.10596631 ... 0.0153026  0.01273764 0.06339555]
 [0.11914773 0.10596631 1.         ... 0.01362021 0.02530203 0.04713654]
 ...
 [0.00920449 0.0153026  0.01362021 ... 1.         0.03580673 0.04301974]
 [0.02822247 0.01273764 0.02530203 ... 0.03580673 1.         0.01853061]
 [0.08639165 0.06339555 0.04713654 ... 0.04301974 0.01853061 1.        ]]


In [10]:
similarity.shape

(14952, 14952)

In [11]:
# getting the anime name from the user

anime_name = input(' Enter your favourite movie name : ')



 Enter your favourite movie name : naruto


In [12]:
# creating a list with all the anime names given in the dataset

list_of_all_titles = df['English name'].tolist()
print(list_of_all_titles)


['Cowboy Bebop', 'Cowboy Bebop:The Movie', 'Trigun', 'Witch Hunter Robin', 'Beet the Vandel Buster', 'Unknown', 'Honey and Clover', 'Unknown', 'Unknown', 'Monster', 'Naruto', 'One Piece', 'The Prince of Tennis', 'Unknown', 'School Rumble', 'Desert Punk', 'Texhnolyze', 'Trinity Blood', 'Yakitate!! Japan', 'Unknown', 'Neon Genesis Evangelion', 'Neon Genesis Evangelion:Death & Rebirth', 'Neon Genesis Evangelion:The End of Evangelion', 'Berserk', 'Ghost in the Shell', 'Samurai X:Trust and Betrayal', 'Rurouni Kenshin', 'Samurai X:The Motion Picture', 'AKIRA', '.hack//Sign', 'Oh! My Goddess', 'Ah! My Goddess', 'Angel Sanctuary', 'Battle Doll Angelic Layer', 'Ai Yori Aoshi', 'Unknown', 'Unknown', 'Unknown', 'Beck:Mongolian Chop Squad', 'Blue Gender', 'Chobits', 'Chrono Crusade', 'D.N.Angel', 'D.C.~Da Capo~', 'DearS', 'Rozen Maiden', 'Rozen Maiden:Träumend', 'Azumanga Daioh:The Animation', 'Basilisk', 'Black Cat', 'Unknown', 'Full Metal Panic!', 'Full Metal Panic? Fumoffu', 'Full Metal Panic! 

In [13]:
# finding the close match for the anime name given by the user

find_close_match = difflib.get_close_matches(anime_name, list_of_all_titles)
print(find_close_match)

['Naruto']


In [14]:
close_match = find_close_match[0]
print(close_match)

Naruto


In [26]:
# getting the index of the anime
index_of_anime=df[df['English name']==close_match].index[0]
print(index_of_anime)

10


In [27]:
similarity[index_of_anime]

array([0.08649276, 0.06100508, 0.09571915, ..., 0.0157472 , 0.03164357,
       0.04289882])

In [28]:
similarity_score=list(enumerate(similarity[index_of_anime]))
# getting a list of similar animes
print(similarity_score)

[(0, np.float64(0.08649276259628455)), (1, np.float64(0.06100507706835022)), (2, np.float64(0.09571914699675184)), (3, np.float64(0.04515399843286629)), (4, np.float64(0.09560538111974431)), (5, np.float64(0.08938982395351319)), (6, np.float64(0.07512029600787733)), (7, np.float64(0.06886246785819877)), (8, np.float64(0.08247035779385786)), (9, np.float64(0.09573915697810371)), (10, np.float64(1.0)), (11, np.float64(0.11783546691481173)), (12, np.float64(0.10802373986311006)), (13, np.float64(0.08164753613383353)), (14, np.float64(0.08747141345601363)), (15, np.float64(0.07470823223944688)), (16, np.float64(0.0796079025637296)), (17, np.float64(0.09097522996503923)), (18, np.float64(0.05029362268211017)), (19, np.float64(0.06524616857808573)), (20, np.float64(0.09240834738072666)), (21, np.float64(0.08287037276853235)), (22, np.float64(0.10109801687249526)), (23, np.float64(0.12921084484381257)), (24, np.float64(0.074472291881298)), (25, np.float64(0.1301933372706377)), (26, np.float64

In [29]:
len(similarity_score)

14952

In [30]:
# sorting the animes based on their similarity score
sorted_similar_animes = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_animes)

[(10, np.float64(1.0)), (1574, np.float64(0.40584679009005037)), (11659, np.float64(0.3931937857530864)), (6996, np.float64(0.3741796009083048)), (9508, np.float64(0.3329893099054956)), (2058, np.float64(0.312157383221764)), (6269, np.float64(0.2737740181659683)), (5302, np.float64(0.27349523654550195)), (6850, np.float64(0.26933806749418726)), (3516, np.float64(0.26575335069167094)), (7504, np.float64(0.25558675775556833)), (557, np.float64(0.24157750434989367)), (3639, np.float64(0.2302180614303986)), (10716, np.float64(0.22483240105268498)), (2267, np.float64(0.2117815660246144)), (4984, np.float64(0.21150892216424355)), (848, np.float64(0.19336504267618823)), (4552, np.float64(0.17682791225105374)), (6303, np.float64(0.1743973183920015)), (10741, np.float64(0.17259324566363002)), (696, np.float64(0.17135526792402048)), (14548, np.float64(0.16875135723477724)), (1240, np.float64(0.16183142472894707)), (414, np.float64(0.16117082987228443)), (2851, np.float64(0.15925171711717237)), (

In [33]:

# print the name of similar animes based on the id

print('Animes suggested for you : \n')

i = 1

for anime in sorted_similar_animes:
  index = anime[0]
  name_from_index = df.iloc[index]["English name"]
  if name_from_index != "Unknown":
    if (i<20):
      print(i, '.',name_from_index)
      i+=1

Animes suggested for you : 

1 . Naruto
2 . Naruto:Shippuden
3 . Boruto:Naruto Next Generations
4 . Road to Ninja:Naruto the Movie
5 . Boruto:Naruto the Movie
6 . Hidden Leaf Village Grand Sports Festival
7 . NARUTO Spin-Off:Rock Lee & His Ninja Pals
8 . Naruto Shippuden:Konoha Gakuen - Special
9 . The Last:Naruto the Movie
10 . Naruto:The Lost Story - Mission:Protect the Waterfall Village
11 . Naruto:Shippuden the Movie 2 -Bonds-
12 . Naruto:Shippuden the Movie
13 . Naruto the Movie 2:Legend of the Stone of Gelel
14 . Chunin Exam on Fire! and Naruto vs. Konohamaru!
15 . The Lost Village
16 . Naruto:Find the Crimson Four-leaf Clover!
17 . Jiang Ziya
18 . Bird of Fire
19 . Naruto the Movie:Ninja Clash in the Land of Snow


#Anime Recommendation system


In [44]:
anime_name = input(' Enter your favourite movie name : ')

list_of_all_titles = df['English name'].tolist()

find_close_match = difflib.get_close_matches(anime_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_anime=df[df['English name']==close_match].index[0]

similarity_score=list(enumerate(similarity[index_of_anime]))

sorted_similar_animes = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Animes suggested for you : \n')

i = 1

for anime in sorted_similar_animes:
  index = anime[0]
  name_from_index = df.iloc[index]["English name"]
  if name_from_index != "Unknown":
    if (i<20):
      print(i, '.',name_from_index)
      i+=1

 Enter your favourite movie name : One piece
Animes suggested for you : 

1 . One Piece
2 . One Piece Film Z
3 . One Piece:Romance Dawn Story
4 . One Piece Film Strong World
5 . One Piece Special:Open Upon the Great Sea! A Father's Huge, HUGE Dream!
6 . Shironeko Project ZERO CHRONICLE
7 . Rennyo and His Mother
8 . Doraemon the Movie:Nobita's Great Adventure in the South Seas
9 . One Piece:Emergency Planning, A Perfect Strategy for the One Piece
10 . One Piece:Defeat the Pirate Ganzack!
11 . Space Adventure Cobra
12 . Legends of the Dark King:A Fist of the North Star Story
13 . Monkey King:Hero Is Back
14 . One Piece:Baron Omatsuri and the Secret Island
15 . Wrath of the Ninja
16 . K:Return of Kings
17 . One Piece:Dead End
18 . Karas
19 . Fafner of the Blue Sky


#🚀🚀🚀🚀